## Approach 1

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

tickers = [
    "RELIANCE", "TCS", "HDFCBANK", "INFY", "ICICIBANK", "HINDUNILVR", "SBIN", "BHARTIARTL",
    "KOTAKBANK", "LT", "AXISBANK", "ITC", "BAJFINANCE", "MARUTI", "ASIANPAINT", "NESTLEIND",
    "TITAN", "ULTRACEMCO", "POWERGRID", "WIPRO", "JSWSTEEL", "SUNPHARMA", "HCLTECH", "ADANIPORTS",
    "GRASIM", "DIVISLAB", "SBILIFE", "BPCL", "M&M", "SHREECEM", "COALINDIA", "ADANIGREEN",
    "ADANIPOWER", "BRITANNIA", "CIPLA", "TECHM", "HEROMOTOCO", "DRREDDY", "EICHERMOT", "GAIL",
    "UPL", "IOC", "INDUSINDBK", "TATAMOTORS"
]

start_date = "2020-01-01"
end_date   = "2025-01-01"

# Container for final views
views_data = []


for ticker in tickers:
    # For NSE in Yahoo Finance, add ".NS"
    yf_ticker = ticker + ".NS"
    try:
        df = yf.download(yf_ticker, start=start_date, end=end_date, progress=False)
        if df.empty:
            print(f"No data for {ticker}")
            continue
        
        
        # Compute annualized returns
        df["Daily Return"] = df["Close"].pct_change()
        mean_daily_ret = df["Daily Return"].mean()
        # Approx. annualization with 252 trading days
        annual_ret = mean_daily_ret * 252  

        # logic:
        # - If baseline annual return is above 0.15, we might reduce it slightly
        # - Otherwise, we add a small alpha
        if annual_ret > 0.15:
            # Very high historical return, so be conservative
            subjective_view = annual_ret - 0.02
        elif annual_ret < 0:
            # Negative or zero baseline, we might see a bounce
            subjective_view = annual_ret + 0.01
        else:
            # Slightly bullish adjustment
            subjective_view = annual_ret + 0.005

        # If volatility is low, we have higher confidence
        # Calculate standard deviation of daily returns, annualized
        daily_vol = df["Daily Return"].std()
        ann_vol = daily_vol * np.sqrt(252)
        
        # confidence as a simple function of annual volatility
        # Lower vol => higher confidence, range 0.5 to 0.95
        min_conf, max_conf = 0.5, 0.95
        # Normalize vol to a 0-1 scale
        # If vol is > 0.4 => set confidence near 0.5
        # If vol is < 0.15 => set confidence near 0.95
        norm_vol = (ann_vol - 0.15) / (0.4 - 0.15)  # scale vol from 0.15 to 0.4
        norm_vol = max(0, min(norm_vol, 1))        
        confidence = max_conf - (max_conf - min_conf) * norm_vol

        # Ensuring confidence is between 0 and 1
        confidence = round(confidence, 2)
        
        # Round subjective view to 4 decimal places
        subjective_view = round(subjective_view, 4)

        views_data.append({
            "Stock": ticker,
            "View": subjective_view,
            "Confidence": confidence
        })
        
        print(f"{ticker}: baseline={annual_ret:.2%}, view={subjective_view:.2%}, conf={confidence}")
    
    except Exception as e:
        print(f"Error for {ticker}: {e}")


views_df = pd.DataFrame(views_data)
views_df.to_csv("views_finetuned.csv", index=False)

print("\n views_finetuned.csv generated.")


## Approach 2

In [ ]:
import pandas as pd
import numpy as np

stocks = [
    'RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'ICICIBANK', 'HINDUNILVR', 'SBIN', 'BHARTIARTL', 
    'KOTAKBANK', 'LT', 'AXISBANK', 'ITC', 'BAJFINANCE', 'MARUTI', 'ASIANPAINT', 'NESTLEIND', 
    'TITAN', 'ULTRACEMCO', 'POWERGRID', 'WIPRO', 'JSWSTEEL', 'SUNPHARMA', 'HCLTECH', 'ADANIPORTS', 
    'GRASIM', 'DIVISLAB', 'SBILIFE', 'BPCL', 'M&M', 'SHREECEM', 'COALINDIA', 'ADANIGREEN', 
    'ADANIPOWER', 'BRITANNIA', 'CIPLA', 'TECHM', 'HEROMOTOCO', 'DRREDDY', 'EICHERMOT', 'GAIL', 
    'UPL', 'IOC', 'INDUSINDBK', 'TATAMOTORS'
]

# Generate random expected returns (between 5% and 15%) and confidence (between 50% and 100%)
np.random.seed(42)  # For reproducibility
views_data = {
    "Stock": stocks,
    "View": np.round(np.random.uniform(0.05, 0.15, len(stocks)), 4),  # Expected return (5% to 15%)
    "Confidence": np.round(np.random.uniform(0.5, 1.0, len(stocks)), 2)  # Confidence (50% to 100%)
}

# Create DataFrame and save to CSV
views_df = pd.DataFrame(views_data)
views_csv_path = "views.csv"
views_df.to_csv(views_csv_path, index=False)

## Market Cap and Close Prices Data

In [ ]:
import pandas as pd
import yfinance as yf
import time

nifty50_symbols = [
    'RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'ICICIBANK',
    'HINDUNILVR', 'SBIN', 'BHARTIARTL', 'HDFC', 'KOTAKBANK',
    'LT', 'AXISBANK', 'ITC', 'BAJFINANCE', 'MARUTI',
    'ASIANPAINT', 'NESTLEIND', 'TITAN', 'ULTRACEMCO', 'POWERGRID',
    'WIPRO', 'JSWSTEEL', 'SUNPHARMA', 'HCLTECH', 'ADANIPORTS',
    'GRASIM', 'DIVISLAB', 'SBILIFE', 'BPCL', 'M&M',
    'SHREECEM', 'COALINDIA', 'ADANIGREEN', 'ADANIPOWER', 'BRITANNIA',
    'CIPLA', 'TECHM', 'HEROMOTOCO', 'DRREDDY', 'EICHERMOT',
    'GAIL', 'UPL', 'IOC', 'INDUSINDBK', 'TATAMOTORS'
]

def fetch_market_caps(symbols):
    market_cap_data = []
    for symbol in symbols:
        ticker_symbol = f"{symbol}.NS"  # NSE stocks use the .NS suffix on Yahoo Finance
        try:
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            mcap = info.get("marketCap", None)
            if mcap is None:
                print(f"Market cap not available for {symbol}")
            else:
                print(f"{symbol}: Market Cap = {mcap}")
            market_cap_data.append({"Symbol": symbol, "MarketCap": mcap})
        except Exception as e:
            print(f"Error fetching market cap for {symbol}: {e}")
            market_cap_data.append({"Symbol": symbol, "MarketCap": None})
        time.sleep(1)  # Delay to avoid rate-limiting
    return pd.DataFrame(market_cap_data)

def fetch_prices(symbols, start_date, end_date):
    stock_data = {}
    for symbol in symbols:
        ticker_symbol = f"{symbol}.NS"
        try:
            ticker = yf.Ticker(ticker_symbol)
            df = ticker.history(start=start_date, end=end_date)
            if not df.empty:
                stock_data[symbol] = df["Close"]
                print(f"Fetched prices for {symbol}")
            else:
                print(f"No price data for {symbol}")
        except Exception as e:
            print(f"Error fetching prices for {symbol}: {e}")
        time.sleep(1)
    return pd.DataFrame(stock_data)

def main():
    # Fetch market cap data and save to mcap.csv
    market_caps_df = fetch_market_caps(nifty50_symbols)
    market_caps_df.to_csv("mcap.csv", index=False)
    print("Market cap data saved to mcap.csv")
    
    # Fetch historical prices from 2020-01-01 to 2025-01-01 and save to prices.csv
    start_date = "2020-01-01"
    end_date = "2025-01-01"
    prices_df = fetch_prices(nifty50_symbols, start_date, end_date)
    prices_df.to_csv("prices.csv")
    print("Price data saved to prices.csv")

if __name__ == "__main__":
    main()
